In [1]:
# Stages of Data Cleaning
# 1. Parse date/time (if time-stamps present)
# 2. Drop un-necessary columns (for machine learning) like ID, etc.
# 3. Feature Engineering (derive new features from existing features)
# 4. Data imputation: Fill the missing values (with mode, median, mean, etc.) OR Drop rows with missing values
# 3. Handling class imbalance: oversampling e.g. SMOTE, ADASYN 
# 5. One Hot Encoding of categorical features
# 6. Normalization (0-1) / Standarization (mean=0, SD=1)
# 7. PDimensionality Reduction: Feature Transformation (PCA/t-SNE) or Feature selection (chiq-square test, RFE, etc. )
# After data cleaning we can perform:
# Data Modeling (Machine Learning + Regularization) with Hyper-parameter Tuning (Grid search)
# Model Evaluation (Accuracy, Precision, F1 score, Confusion Matrix, AUC) and Visualization

In [2]:
import pandas as pd
import numpy as np
train_df = pd.read_csv("titanic_train.csv")
test_df = pd.read_csv("titanic_test.csv")
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# check for missing values in all the columns
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [5]:
embarked_mode = train_df['Embarked'].mode()
data = [train_df, test_df]
for dataset in data:
    dataset['Embarked'] = dataset['Embarked'].fillna(embarked_mode)

In [6]:
data = [train_df, test_df]
for dataset in data:
    mean = train_df["Age"].mean()
    std = test_df["Age"].std()
    is_null = dataset["Age"].isnull().sum()
    # compute random numbers between the mean, std and is_null
    rand_age = np.random.randint(mean - std, mean + std, size = is_null)
    # fill NaN values in Age column with random values generated
    age_slice = dataset["Age"].copy()
    age_slice[np.isnan(age_slice)] = rand_age
    dataset["Age"] = age_slice
    dataset["Age"] = train_df["Age"].astype(int)


In [7]:
data = [train_df, test_df]
for dataset in data:
    dataset['relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['relatives'] > 0, 'travelled_alone'] = 'No'
    dataset.loc[dataset['relatives'] == 0, 'travelled_alone'] = 'Yes'

In [8]:
for dataset in data:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head(5).transpose()

,0,1,2,3,4
PassengerId,1,2,3,4,5
Survived,0,1,1,1,0
Pclass,3,1,3,1,3
Name,"Braund, Mr. Owen Harris","Cumings, Mrs. John Bradley (Florence Briggs Th...","Heikkinen, Miss. Laina","Futrelle, Mrs. Jacques Heath (Lily May Peel)","Allen, Mr. William Henry"
Sex,male,female,female,female,male
Age,1,2,1,2,2
SibSp,1,1,0,1,0
Parch,0,0,0,0,0
Ticket,A/5 21171,PC 17599,STON/O2. 3101282,113803,373450
Fare,7.25,71.2833,7.925,53.1,8.05


In [9]:
train_df = train_df.drop(['PassengerId','Name','Ticket','Cabin','SibSp','Parch'], axis=1)
train_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,relatives,travelled_alone
0,0,3,male,1,7.2500,S,1,No
1,1,1,female,2,71.2833,C,1,No
2,1,3,female,1,7.9250,S,0,Yes
3,1,1,female,2,53.1000,S,1,No
4,0,3,male,2,8.0500,S,0,Yes


In [10]:
test_df = test_df.drop(['Name','Ticket','Cabin','SibSp','Parch'], axis=1)
test_df.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,relatives,travelled_alone
0,892,3,male,1,7.8292,Q,0,Yes
1,893,3,female,2,7.0000,S,1,No
2,894,2,male,1,9.6875,Q,0,Yes
3,895,3,male,2,8.6625,S,0,Yes
4,896,3,female,2,12.2875,S,2,No


In [11]:
numerical_features = list(train_df.select_dtypes(include=['int64', 'float64', 'int32']).columns)
numerical_features

['Survived', 'Pclass', 'Age', 'Fare', 'relatives']

In [12]:
del numerical_features[0]
numerical_features

['Pclass', 'Age', 'Fare', 'relatives']

In [13]:
from sklearn.preprocessing import StandardScaler
ss_scaler = StandardScaler()
train_df_ss = pd.DataFrame(data = train_df)
train_df_ss[numerical_features] = ss_scaler.fit_transform(train_df_ss[numerical_features])
train_df_ss.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,relatives,travelled_alone
0,0,0.827377,male,-0.150843,-0.502445,S,0.059160,No
1,1,-1.566107,female,-0.019464,0.786845,C,0.059160,No
2,1,0.827377,female,-0.150843,-0.488854,S,-0.560975,Yes
3,1,-1.566107,female,-0.019464,0.420730,S,0.059160,No
4,0,0.827377,male,-0.019464,-0.486337,S,-0.560975,Yes


In [14]:
encode_col_list = list(train_df.select_dtypes(include=['object']).columns)
encode_col_list

['Sex', 'Embarked', 'travelled_alone']

In [15]:
for i in encode_col_list:
    train_df_ss = pd.concat([train_df_ss,pd.get_dummies(train_df_ss[i], prefix=i)],axis=1)
    train_df_ss.drop(i, axis = 1, inplace=True)
train_df_ss.head()

,Survived,Pclass,Age,Fare,relatives,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,travelled_alone_No,travelled_alone_Yes
0,0,0.827377,-0.150843,-0.502445,0.059160,0,1,0,0,1,1,0
1,1,-1.566107,-0.019464,0.786845,0.059160,1,0,1,0,0,1,0
2,1,0.827377,-0.150843,-0.488854,-0.560975,1,0,0,0,1,0,1
3,1,-1.566107,-0.019464,0.420730,0.059160,1,0,0,0,1,1,0
4,0,0.827377,-0.019464,-0.486337,-0.560975,0,1,0,0,1,0,1


In [16]:
numerical_features = list(test_df.select_dtypes(include=['int64', 'float64', 'int32']).columns)
del numerical_features[0]
ss_scaler = StandardScaler()
test_df_ss = pd.DataFrame(data = test_df)
test_df_ss[numerical_features] = ss_scaler.fit_transform(test_df_ss[numerical_features])
test_df_ss.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,relatives,travelled_alone
0,892,0.873482,male,-0.143799,-0.497023,Q,-0.553443,Yes
1,893,0.873482,female,-0.005620,-0.511885,S,0.105643,No
2,894,-0.315819,male,-0.143799,-0.463715,Q,-0.553443,Yes
3,895,0.873482,male,-0.005620,-0.482087,S,-0.553443,Yes
4,896,0.873482,female,-0.005620,-0.417112,S,0.764728,No


In [17]:
test_encode_col_list = list(test_df.select_dtypes(include=['object']).columns)
for i in test_encode_col_list:
    test_df_ss = pd.concat([test_df_ss,pd.get_dummies(test_df_ss[i], prefix=i)],axis=1)
    test_df_ss.drop(i, axis = 1, inplace=True)
test_df_ss.head()

,PassengerId,Pclass,Age,Fare,relatives,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,travelled_alone_No,travelled_alone_Yes
0,892,0.873482,-0.143799,-0.497023,-0.553443,0,1,0,1,0,0,1
1,893,0.873482,-0.005620,-0.511885,0.105643,1,0,0,0,1,1,0
2,894,-0.315819,-0.143799,-0.463715,-0.553443,0,1,0,1,0,0,1
3,895,0.873482,-0.005620,-0.482087,-0.553443,0,1,0,0,1,0,1
4,896,0.873482,-0.005620,-0.417112,0.764728,1,0,0,0,1,1,0


In [18]:
X_train = train_df_ss.drop("Survived", axis=1)
Y_train = train_df_ss["Survived"]
X_test  = test_df_ss.drop("PassengerId", axis=1).copy()

In [19]:
#X_test.to_csv("abc.csv")

In [20]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()  # Fit our model to the training data
logreg.fit(X_train, Y_train)   # Predict on the test data
logreg_predictions = logreg.predict(X_test)
logreg_predictions

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [21]:
logreg_data = pd.read_csv('titanic_test.csv')
logreg_data.insert((logreg_data.shape[1]),'Survived',logreg_predictions)
logreg_data
#logreg_data.to_csv('LogisticRegression_SS_OH_FE2.csv')

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,0
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,1
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,0
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,0
